In [12]:
import pandas as pd

In [157]:
# Data from InsideAirBNB

# http://data.insideairbnb.com/france/ile-de-france/paris/2017-04-04/data/calendar.csv.gz
df_calendar = pd.read_csv('Downloads/calendar.csv')
# http://data.insideairbnb.com/france/ile-de-france/paris/2017-04-04/data/listings.csv.gz
df_listing = pd.read_csv('Downloads/listings.csv')

df_calendar.columns = ['id', 'date', 'available', 'price']
df_calendar['date'] = pd.to_datetime(df_calendar['date'], format='%Y-%m-%d')

df_calendar = df_calendar.loc[df_calendar['available'] == 'f']

agg_duration = df_calendar.groupby(['id']).size().reset_index()
agg_duration.columns = ['id', 'duration']

df_final = df_listing.merge(agg_duration, on='id', how='left')
#df_final.to_csv('Downloads/offre_durationbyID.csv', sep=";", index=False)

/Users/wngauv/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (43,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [175]:
# http://data.insideairbnb.com/france/ile-de-france/paris/2017-04-04/visualisations/listings.csv
df_host = pd.read_csv('Downloads/listings-2.csv')
agg_host = df_host.groupby(['host_id']).size().reset_index()
agg_host = agg_host.rename(columns={0:'rent_count'})
agg_host.sort_values('rent_count', ascending=False, inplace=True)
#agg_host.to_csv('Downloads/rentcount_per_host.csv', sep=";")

In [14]:
# Data from DataGouv
# https://www.data.gouv.fr/fr/datasets/donnees-de-lencadrement-des-loyers-a-paris-et-lille/
df_loyer = pd.read_csv('Downloads/encadrement_loyers.csv')
df_loyer = df_loyer.loc[(df_loyer['ville'] == 'PARIS') &
                        (df_loyer['annee'] >= 2017) & 
                        (df_loyer['meuble'] == 1)]
df_loyer_par_quartier = df_loyer.groupby(['nom_quartier'])['ref'].mean().reset_index()
df_loyer_par_quartier['nom_quartier'] = df_loyer_par_quartier['nom_quartier'].str.lower() \
                                                    .str.normalize('NFKD') \
                                                    .str.encode('ascii', errors='ignore') \
                                                    .str.decode('utf-8')
df_loyer_par_quartier['nom_quartier'] = df_loyer_par_quartier['nom_quartier'].replace(
                                                                            [r'saint-',
                                                                             r'sainte-',
                                                                             r'faubourg-',
                                                                             r'avoie',
                                                                             r'^notre-dame-',
                                                                             r'^parc-de-',
                                                                             r' 15art',
                                                                             r'^la ',
                                                                             r'plaine de '],
                                                                            ['st-',
                                                                             'ste-',
                                                                             'fg-',
                                                                             'avoye',
                                                                             'nd-',
                                                                             'parc-',
                                                                             '',
                                                                             '',
                                                                             'plaine-'],
                                                                            regex=True)

In [15]:
# Data from OpenStreetMap
df_quartier = pd.read_csv('Downloads/osm_paris_quartiers.csv', sep=";", usecols=["osm_id", "name",
                                     "addr.postcode", "admin_level",
                                     "alt_name", "boundary",
                                     "ref.INSEE", "short_name", "type",
                                     "wikidata", "wikipedia"])
df_quartier = df_quartier.loc[~df_quartier['short_name'].isnull()]
df_quartier['short_name'] = df_quartier['short_name'].str.lower() \
                                                    .str.normalize('NFKD') \
                                                    .str.encode('ascii',
                                                                errors='ignore') \
                                                    .str.decode('utf-8')
df_quartier['short_name'] = df_quartier['short_name'].replace([r'^le ', r'^la ', r'les '],
                                                              '',
                                                              regex=True)
df_quartier['short_name'] = df_quartier['short_name'].replace(
                                                        ['enfants rouges',
                                                         r' des ',
                                                         'palais royal',
                                                         'vendome',
                                                         r'monceau'],
                                                        ['enfants-rouges',
                                                         '-des-',
                                                         'palais-royal',
                                                         'place-vendome',
                                                         'monceaux'],
                                                        regex=True)
df_quartier = df_quartier.rename(columns={'short_name':'nom_quartier'})

FileNotFoundError: File b'.Downloads/osm_paris_quartiers.csv' does not exist

In [152]:
#df_quartier.merge(df_loyer_par_quartier, on="nom_quartier").to_csv('Downloads/loyer_par_quartier.csv', sep=";", index=False)

In [149]:
set(df_quartier['nom_quartier'].unique().tolist()) - set(df_loyer_par_quartier['nom_quartier'].unique().tolist())

{'iles'}

In [150]:
set(df_loyer_par_quartier['nom_quartier'].unique().tolist()) - set(df_quartier['nom_quartier'].unique().tolist())

{'fg-du-roule', 'notre-dame'}

In [6]:
import pandas as pd

df = pd.read_csv('/Users/wngauv/Downloads/laposte_hexasmal.csv', sep=';', usecols=['insee', 'commune', 'code_postal', 'libelle', ''])

TypeError: parser_f() got an unexpected keyword argument 'index'

In [9]:
df.loc[df['BEAUREGARD'].str.contains('MONACO')]

,1030,BEAUREGARD,1480,BEAUREGARD.1,"46.0178170669, 4.81482624573"
14736,99138,MONACO,98000,MONACO,NaN
